In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import os
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim
from torchvision import datasets, transforms, models
from matplotlib import pyplot as plt
import pandas as pd
from efficientnet_pytorch import EfficientNet

In [ ]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Define Directories

In [ ]:
train_folder = "/kaggle/input/fake-or-real-dataset/train/"
val_folder = "/kaggle/input/fake-or-real-dataset/test/"
epochs = 100
lr = 0.003

Prepare Datasets

In [ ]:
# Define transforms
transform_enb3 = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
full_data = datasets.ImageFolder(root=train_folder, transform=transform_enb3)
full_data.classes[0] = 1 # set fake as 1
full_data.classes[1] = 0 # set real as 0

val_data = datasets.ImageFolder(root=val_folder, transform=transform_enb3)
val_data.classes[0] = 1 # set fake as 1

# Define train-test split ratio
train_size = int(0.8 * len(full_data))
test_size = len(full_data) - train_size

# Split the dataset
train_data, test_data = torch.utils.data.random_split(full_data, [train_size, test_size])

# Create DataLoader for each dataset
batch_size = 256
train_loader = DataLoader(train_data, batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size, shuffle=False)

# Print number of samples in each split
print(f'Training samples: {len(train_data)}')
print(f'Testing samples: {len(test_data)}')

testing dataloaders with 5 random photos

In [ ]:
i = 0
for images, labels in train_loader:  # Assuming train_loader gives a batch of images and labels
    if i >= 5:
        break
    
    # Access the first image in the batch and permute dimensions
    image = images[0].permute(1, 2, 0)
    
    # Display the image
    plt.imshow(image)
    plt.title("real" if labels[0].item() == 1 else "fake")
    plt.show()
    
    i += 1

In [ ]:
from tqdm import tqdm
# Training loop
def train_model(model, criterion, optimizer, dataloaders, classification_layer,finetuning_lr,num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            running_loss = 0.0
            running_corrects = 0
            
            # Use tqdm to wrap the dataloader
            phase_dataloader = tqdm(dataloaders[phase], desc=phase)
            for inputs, labels in phase_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device).float().unsqueeze(1)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward
                if phase == "train":
                    if epoch >= 0.8 * num_epochs:
                        # Unfreeze all layers for fine-tuning
                        for param in model.parameters():
                            param.requires_grad = True
                        optimizer.param_groups[0]['lr'] = finetuning_lr
                    else:
                        # Freeze the CNN part
                        for param in model.parameters():
                            param.requires_grad = False
                        # Unfreeze the classification layer
                        for param in classification_layer.parameters():
                            param.requires_grad = True
                elif phase == "val":
                    for param in model.parameters():
                            param.requires_grad = False

                        
                outputs = model(inputs)
                preds = torch.sigmoid(outputs)
                preds = (preds > 0.5).float()
#                 print(preds)
                loss = criterion(outputs, labels)

                # Backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        print()

    return model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def test_model(model, dataloader, criterion, device):
    model.eval()  # Set model to evaluation mode
    running_loss = 0.0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        val_dataloader = tqdm(dataloader, desc="validation")
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            labels = torch.ones(outputs.size()).to(device)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)

            preds = (torch.sigmoid(outputs) > 0.5) * 1
#             print(preds)
            all_labels.append(labels)
            all_preds.append(preds)

    # Move all predictions and labels to CPU once
    all_labels = torch.cat(all_labels).cpu().numpy()
    all_preds = torch.cat(all_preds).cpu().numpy()

    epoch_loss = running_loss / len(dataloader.dataset)
#     print(all_labels)
#     print(all_preds)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f'Test Loss: {epoch_loss:.4f}')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

    return epoch_loss, accuracy, precision, recall, f1

In [ ]:
import datetime as dt
def save_model(model, model_name,model_evaluation_history):
    model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history
    # Define the string date format.
    # Get the current Date and Time in a DateTime Object.
    # Convert the DateTime object to string according to the style mentioned in date_time_format string.
    date_time_format = '%Y_%m_%d__%H_%M_%S'
    current_date_time_dt = dt.datetime.now()
    current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

    # Define a useful name for our model to make it easy for us while navigating through multiple saved models.
    model_file_name = f'{model_name}___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.pth'

    # Save your Model.
    torch.save(model.state_dict(), f'{model_file_name}_entire_dict')
    torch.save(model, f'{model_file_name}_entire')

## **EfficientNet Training**

In [ ]:

model = efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
dataloaders = {'train' : train_loader, 'val' : test_loader}
device = "cuda"
# Modify the classifier to fit the binary classification problem
num_features = model.classifier.fc.in_features
model.classifier.fc = nn.Linear(num_features, 1)  # Output layer for binary classification

# Move the model to the appropriate device
model = model.to(device)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy loss with logits
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model = train_model(model, criterion, optimizer, dataloaders, model.classifier.fc ,10e-6)

epoch_loss, accuracy, precision, recall, f1 = test_model(model, val_loader, criterion, device)

save_model(model, "efficientnet-B0", (epoch_loss, accuracy))


## **ResNet Training**

In [ ]:
model = models.resnet18(pretrained=True)
dataloaders = {'train' : train_loader, 'val' : test_loader}
device = "cuda"
# Modify the classifier to fit the binary classification problem
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)  # Output layer for binary classification

# Move the model to the appropriate device
model = model.to(device)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy loss with logits
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model = train_model(model, criterion, optimizer, dataloaders,model.fc ,10e-6)

epoch_loss, accuracy, precision, recall, f1 = test_model(model, val_loader, criterion, device)

save_model(model, "ResNet-18", (epoch_loss, accuracy))

## **ResNext**

In [ ]:
# Define the dataloaders
dataloaders = {'train': train_loader, 'val': test_loader}

# Set device to GPU if available
device = torch.device("cuda")

# Load the ResNeXt model with pretrained weights
model = models.resnext50_32x4d(pretrained=True)

# Modify the classifier to fit the binary classification problem
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)  # Output layer for binary classification

# Move the model to the appropriate device
model = model.to(device)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy loss with logits
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model = train_model(model, criterion, optimizer, dataloaders, model.fc, 10e-6, num_epochs=2)

epoch_loss, accuracy, precision, recall, f1 = test_model(model, val_loader, criterion, device)

save_model(model, "resnext50_32x4d", (epoch_loss, accuracy))

## **VGG**

In [ ]:
# Define the dataloaders
dataloaders = {'train': train_loader, 'val': test_loader}

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the VGG model with pretrained weights
model = models.vgg16(pretrained=True)

# Modify the classifier to fit the binary classification problem
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 1)  # Output layer for binary classification

# Move the model to the appropriate device
model = model.to(device)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy loss with logits
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model = train_model(model, criterion, optimizer, dataloaders, model.classifier[6], 10e-6)

epoch_loss, accuracy, precision, recall, f1 = test_model(model, val_loader, criterion, device)

save_model(model, "vgg16", (epoch_loss, accuracy))